In [11]:
import pandas as pd
import praw 
import sqlite3

In [12]:
df = pd.read_csv('/Users/jacksonsorenson/Documents/Computational Media Lab/Original CSV/VAMoS - Reddit Handles _ Verified and Public.csv')
df.head()

,ResponseId,REDDIT
0,R_0369xXP09aPOajv,@dj6eyes
1,R_10VkQQQBDp38cS4,Fenveon11037
2,R_1CfDJYE2y1wwPDd,HowSheGoinEh02
3,R_1dH45I2SxKxpLDA,CursivePluto
4,R_1eEIL6jLKSiCaNq,Yexai_


In [13]:
# Connecting to Reddit API with Praw
reddit = praw.Reddit(
    client_id= "4Fj-6rBvYvhjEOb_3xw6PA",
    client_secret= "t83zQerjf3UZjUMmV5DiPZcuFOtM5g",
    user_agent= "Vaping Reddit Scrape by Jackson Sorenson"   )

In [14]:
#Creating a 'comments' data base to mine user comments to show  username, their comment, created id, subreddit title, and the amount of upvotes they got (socre).

conn = sqlite3.connect('comment_data.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS comments (
    username TEXT,
    body TEXT,
    created_utc INTEGER,
    subreddit TEXT,
    score INTEGER
)
''')

conn.commit()
conn.close()

In [15]:
# Connecting to data base and beginning to mine

conn = sqlite3.connect('comment_data.db')
cursor = conn.cursor()

for username in df["REDDIT"]:
    try:
        user = reddit.redditor(username)
        
        # for the 'limit' you need to set it to "None" or a high number. Praw will default to only mine the most recent submission for each user. 
        for comment in user.comments.new(limit=1000):  
            print(comment.id, comment.body, comment.created_utc, comment.subreddit.display_name, comment.score)
            
            cursor.execute('''
                INSERT INTO comments (username, body, created_utc, subreddit, score)
                VALUES (?, ?, ?, ?, ?)
            ''', (username, comment.body, comment.created_utc, comment.subreddit.display_name, comment.score))
            
    except Exception as e:
        print(f"Could not find data for user {username}: {e}")

# After every use, make sure to commit and close the cdatabase. I had a lot of trouble with not closing it each time earlier.
conn.commit()
conn.close() 

Could not find data for user @dj6eyes: received 404 HTTP response
ljz8739 Forgot to add this but if you need help making the papercraft here’s a video I used when making my first ones [Papercraft Tutorial](https://youtu.be/F-S_oDVQbog?si=mRyyPuRMqPwuSRFf) 1724655949.0 KieranDefenseSquad 1
ljttmg1 Yes I make papercrafts so that people can their own tiny ver of the characters so I would love it if you used it ╰(*´︶`*)╯♡ 1724572842.0 KieranDefenseSquad 3
kzgsvqk Friend Code: 6P8GJH
Thank you in advance for using my code 1713055128.0 NuCarnival 1
j9mqp2w It’d be cool to win 1677117990.0 onewheel 1
iz5zw4k Your netid, which is the first part of your email before the @, and the password for your email/netid 1670353221.0 utarlington 2
iwmbicw FedEx and UPS ship fast but the prices are very high. Sealine is a good affordable option but shipping time is between 1 to 2 months. 1668623076.0 repbudgetsneakers 1
iul0dyb For sealine it’s volumetric but cheap af bc it’s through sea, just make sure to

In [17]:
# Check to see if the data is stored in 'comments' databse.

conn = sqlite3.connect('comment_data.db')
df_comments = pd.read_sql_query("SELECT * FROM comments", conn)
df_comments.head()

,username,body,created_utc,subreddit,score
0,Fenveon11037,Forgot to add this but if you need help making...,1724655949,KieranDefenseSquad,1
1,Fenveon11037,Yes I make papercrafts so that people can thei...,1724572842,KieranDefenseSquad,3
2,Fenveon11037,Friend Code: 6P8GJH\nThank you in advance for ...,1713055128,NuCarnival,1
3,CursivePluto,It’d be cool to win,1677117990,onewheel,1
4,CursivePluto,"Your netid, which is the first part of your em...",1670353221,utarlington,2


In [18]:
# Turn to CSV

df_comments.to_csv('comment_data.csv', index=False)